# tensorFlow物件偵測的 Demo

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
print('成功import12')

成功import12


## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
print('env ok')

env ok


## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util
# 內含有將物件偵測結果視覺化的API
from utils import visualization_utils as vis_util
print('ok')

ok


# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
# TF已經訓練好的模型名稱
MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
# TF已經訓練好的模型檔案(本身是一個tar.gz的壓縮檔)
MODEL_FILE = MODEL_NAME + '.tar.gz'

# 這個TF訓練好模型 所在網站的目錄名稱
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# TF實際用來做物件偵測的模型 
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# 這個模型對應到的種類txt檔，內涵每個種類的索引&名稱
# 之後物件偵測得到的資訊會是索引，所以需要此文件參考名稱
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

# 預設模型種類數量 為90種
NUM_CLASSES = 90
print('Model preparation is ok...')

Model preparation is ok...


## Download Model (如果之前下載過模組，那該這個區塊可以不用再執行一次)

In [5]:
# opener = urllib.request.URLopener()
# # opener.retrieve(下載的檔案來源, 下載的檔案存放的本地位址)
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#     file_name = os.path.basename(file.name)
#     if 'frozen_inference_graph.pb' in file_name:
#         tar_file.extract(file, os.getcwd())
# print('Download Model is ok', os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names,

so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  

Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [7]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = (label_map_util.convert_label_map_to_categories(
    label_map, 
    max_num_classes=NUM_CLASSES, 
    use_display_name=True))

category_index = label_map_util.create_category_index(categories)
print(category_index)

{1: {'id': 1, 'name': 'person'}, 2: {'id': 2, 'name': 'bicycle'}, 3: {'id': 3, 'name': 'car'}, 4: {'id': 4, 'name': 'motorcycle'}, 5: {'id': 5, 'name': 'airplane'}, 6: {'id': 6, 'name': 'bus'}, 7: {'id': 7, 'name': 'train'}, 8: {'id': 8, 'name': 'truck'}, 9: {'id': 9, 'name': 'boat'}, 10: {'id': 10, 'name': 'traffic light'}, 11: {'id': 11, 'name': 'fire hydrant'}, 13: {'id': 13, 'name': 'stop sign'}, 14: {'id': 14, 'name': 'parking meter'}, 15: {'id': 15, 'name': 'bench'}, 16: {'id': 16, 'name': 'bird'}, 17: {'id': 17, 'name': 'cat'}, 18: {'id': 18, 'name': 'dog'}, 19: {'id': 19, 'name': 'horse'}, 20: {'id': 20, 'name': 'sheep'}, 21: {'id': 21, 'name': 'cow'}, 22: {'id': 22, 'name': 'elephant'}, 23: {'id': 23, 'name': 'bear'}, 24: {'id': 24, 'name': 'zebra'}, 25: {'id': 25, 'name': 'giraffe'}, 27: {'id': 27, 'name': 'backpack'}, 28: {'id': 28, 'name': 'umbrella'}, 31: {'id': 31, 'name': 'handbag'}, 32: {'id': 32, 'name': 'tie'}, 33: {'id': 33, 'name': 'suitcase'}, 34: {'id': 34, 'name'

## Helper code

In [8]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [17]:
import time 

# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'

TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'dog{}.jpg'.format(i)) for i in range(0, 100) ]
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [28]:
import time

image_obj_data = []

with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    for image_path in TEST_IMAGE_PATHS:
      tStart = time.time()
      image = Image.open(image_path)
      # the array based representation of the image will be used later in order to prepare the
      # result image with boxes and labels on it.
      image_np = load_image_into_numpy_array(image)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      # Each box represents a part of the image where a particular object was detected.
      boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      # Each score represent how level of confidence for each of the objects.
      # Score is shown on the result image, together with the class label.
      scores = detection_graph.get_tensor_by_name('detection_scores:0')
      classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
      # Actual detection.
      (boxes, scores, classes, num_detections) = sess.run(
          [boxes, scores, classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      
#     # 印出每個物件的位置.分數.種類
#       print('boxes[0]:', boxes[0])
#       print('scores[0]:', scores[0])
#       print('classes:', classes)
#       print('\n\n\n')
      for i in range(2):
          print("-----------------------------------------------------")
      print('---------------- %s--------------' % image_path)      
      for i in range(2):
        print("-----------------------------------------------------")
      # 分數高於0.5的才可以視為相符
      obj_length = len([x for x in scores[0] if x > 0.5])
      width, height = image.size
    
      # 第一實驗魔改_Begin 該圖片所有的物件種類   
      all_objs = classes[0]
      ok_objs = all_objs[0:obj_length]
      objs_classes = set(ok_objs)
    
      # Result 1 對應原始物件索引陣列的字典(Key為類別名， Value為對應原始陣列的物件索引陣列)
      img_classes = dict()

      for obj_class in objs_classes:
        img_classes[category_index[obj_class]['name']] = len(np.nonzero(ok_objs == obj_class)[0])
        print(category_index[obj_class]['name'], ':', len(np.nonzero(ok_objs == obj_class)[0]))
#         print('種類:', category_index[obj_class]['name'])
#         print(np.nonzero(ok_objs == obj_class)[0], '有', len(np.nonzero(ok_objs == obj_class)[0]),'個') 

#       # Result 2 原始物件的標記位置陣列[(x1, y1, x2, y2)]
#       img_obj_coordinates = boxes[0][0:obj_length]
        
#       # Result 3 原始物件的相似分數陣列
#       img_obj_scores = scores[0][0:obj_length]

      # 標示每個圖片上所有的物件Visualization of the results of a detection.
    
#       # 如果要顯示圖片在ipynb，這個區塊可以取消註解 
#       vis_util.visualize_boxes_and_labels_on_image_array(
#           image_np,
#           np.squeeze(boxes),
#           np.squeeze(classes).astype(np.int32),
#           np.squeeze(scores),
#           category_index,
#           use_normalized_coordinates=True,
#           line_thickness=4)
#       plt.figure(figsize=IMAGE_SIZE)
#       plt.imshow(image_np)
#       # 如果要顯示圖片在ipynb，這個區塊可以取消註解 
      # 第一實驗魔改_End
      image_obj_data.append(img_classes)
      tEnd = time.time()
      print('每個圖片建立檔案結束:', tEnd - tStart)
      if len(image_obj_data) % 10 == 0:
        print('已處理', len(image_obj_data), '張...')

-----------------------------------------------------
-----------------------------------------------------
---------------- test_images/dog0.jpg--------------
-----------------------------------------------------
-----------------------------------------------------
person : 1
dog : 1
每個圖片建立檔案結束: 3.0085480213165283
-----------------------------------------------------
-----------------------------------------------------
---------------- test_images/dog1.jpg--------------
-----------------------------------------------------
-----------------------------------------------------
person : 2
dog : 1
car : 1
每個圖片建立檔案結束: 0.38486599922180176
-----------------------------------------------------
-----------------------------------------------------
---------------- test_images/dog2.jpg--------------
-----------------------------------------------------
-----------------------------------------------------
每個圖片建立檔案結束: 0.1586301326751709
-----------------------------------------------------
--

-----------------------------------------------------
-----------------------------------------------------
---------------- test_images/dog26.jpg--------------
-----------------------------------------------------
-----------------------------------------------------
person : 1
dog : 1
每個圖片建立檔案結束: 0.33687686920166016
-----------------------------------------------------
-----------------------------------------------------
---------------- test_images/dog27.jpg--------------
-----------------------------------------------------
-----------------------------------------------------
dog : 1
每個圖片建立檔案結束: 0.3818631172180176
-----------------------------------------------------
-----------------------------------------------------
---------------- test_images/dog28.jpg--------------
-----------------------------------------------------
-----------------------------------------------------
person : 1
dog : 1
每個圖片建立檔案結束: 0.1910390853881836
-----------------------------------------------------

-----------------------------------------------------
-----------------------------------------------------
---------------- test_images/dog53.jpg--------------
-----------------------------------------------------
-----------------------------------------------------
person : 1
dog : 1
每個圖片建立檔案結束: 0.24103808403015137
-----------------------------------------------------
-----------------------------------------------------
---------------- test_images/dog54.jpg--------------
-----------------------------------------------------
-----------------------------------------------------
每個圖片建立檔案結束: 0.1753079891204834
-----------------------------------------------------
-----------------------------------------------------
---------------- test_images/dog55.jpg--------------
-----------------------------------------------------
-----------------------------------------------------
person : 1
dog : 1
每個圖片建立檔案結束: 0.3648519515991211
-----------------------------------------------------
-------

-----------------------------------------------------
-----------------------------------------------------
---------------- test_images/dog80.jpg--------------
-----------------------------------------------------
-----------------------------------------------------
dog : 1
每個圖片建立檔案結束: 0.3942599296569824
-----------------------------------------------------
-----------------------------------------------------
---------------- test_images/dog81.jpg--------------
-----------------------------------------------------
-----------------------------------------------------
dog : 1
每個圖片建立檔案結束: 0.3534719944000244
-----------------------------------------------------
-----------------------------------------------------
---------------- test_images/dog82.jpg--------------
-----------------------------------------------------
-----------------------------------------------------
person : 1
dog : 1
每個圖片建立檔案結束: 0.2075788974761963
-----------------------------------------------------
-----------

In [27]:
# 實驗2
# 此為一對多的實驗
# 讀取一個Query(Ex: 1 個Person, 2個Dog)，從20張圖片挑出符合Query條件的圖片。

tStart = time.time()

# A組圖代表 前75個 圖片的物件字典
a_obj_data = image_obj_data[0:75]
# B組圖代表 後25個 圖片的物件字典
b_obj_data = image_obj_data[75:100]

# query = {'kite':4,'person': 4}
# query = {'dog': 2}
print('-----------------')

# result，為B組每一張圖片跟A組物件數比對成功串列的串列。

result = list()

for b_key, b_objs in enumerate(b_obj_data):
    b_match_a = list()
    for a_key, a_objs in enumerate(a_obj_data):
        if b_objs == a_objs:
            b_match_a.append(a_key)
    print('b組第',b_key,'張圖跟A組以下配對成功:')
    print(b_match_a)
    # 即使沒有配對成功也會添加到result，只不過會是空list而已
    result.append(b_match_a)

tEnd = time.time()
print('第二實驗_比對結束:', tEnd - tStart)
print(result)

-----------------
b組第 0 張圖跟A組以下配對成功:
[17, 18]
b組第 1 張圖跟A組以下配對成功:
[17, 18]
b組第 2 張圖跟A組以下配對成功:
[]
b組第 3 張圖跟A組以下配對成功:
[11, 12, 13, 15, 16, 46, 60, 73]
b組第 4 張圖跟A組以下配對成功:
[]
b組第 5 張圖跟A組以下配對成功:
[3, 5, 7, 10, 19, 21, 23, 27, 29, 33, 35, 39, 40, 42, 44, 52, 56, 58, 64, 66, 72]
b組第 6 張圖跟A組以下配對成功:
[3, 5, 7, 10, 19, 21, 23, 27, 29, 33, 35, 39, 40, 42, 44, 52, 56, 58, 64, 66, 72]
b組第 7 張圖跟A組以下配對成功:
[0, 6, 20, 24, 26, 28, 31, 34, 38, 45, 47, 49, 53, 55, 62, 63, 65, 67, 70]
b組第 8 張圖跟A組以下配對成功:
[]
b組第 9 張圖跟A組以下配對成功:
[3, 5, 7, 10, 19, 21, 23, 27, 29, 33, 35, 39, 40, 42, 44, 52, 56, 58, 64, 66, 72]
b組第 10 張圖跟A組以下配對成功:
[]
b組第 11 張圖跟A組以下配對成功:
[17, 18]
b組第 12 張圖跟A組以下配對成功:
[0, 6, 20, 24, 26, 28, 31, 34, 38, 45, 47, 49, 53, 55, 62, 63, 65, 67, 70]
b組第 13 張圖跟A組以下配對成功:
[]
b組第 14 張圖跟A組以下配對成功:
[0, 6, 20, 24, 26, 28, 31, 34, 38, 45, 47, 49, 53, 55, 62, 63, 65, 67, 70]
b組第 15 張圖跟A組以下配對成功:
[0, 6, 20, 24, 26, 28, 31, 34, 38, 45, 47, 49, 53, 55, 62, 63, 65, 67, 70]
b組第 16 張圖跟A組以下配對成功:
[4]
b組第 17 張圖跟A組以下配對成功:
[3, 5, 